In [1]:
import findspark
findspark.init()

## Creating the Graph

In [2]:
#CSV takes the file and creates and RDD of character,issue
csv=(sc.textFile('source.csv').map(lambda x: x.split('","'))
                                .map(lambda x:[x[0][1:],x[1][:-1]]))
csv.cache()
#issues_by_character groups the issues with character as key
issues_by_character=csv.groupByKey().map(lambda x:[x[0],list(x[1])])
#characters_by_issue creates a dict with issue as key and list of chars as value
characters_by_issue=csv.map(lambda x:[x[1],x[0]]).groupByKey().map(lambda x:[x[0],list(x[1])]).collectAsMap()
#linked characters is an RDD that has the x[0] as the char name and a list of chars its linked to as x[1]
linked_chars=(issues_by_character
              .map(lambda x:[x[0],[characters_by_issue[issue] for issue in x[1]]])
              .map(lambda x:[x[0],[char for chars in x[1] for char in chars]])
              .map(lambda x:[x[0],list(set(x[1]))])
              .map(lambda x:[x[0],[a for a in x[1] if a != x[0]]]))
              
linked_chars.cache()

PythonRDD[12] at RDD at PythonRDD.scala:43

##Breadth-first search on graph

In [351]:
accum = sc.accumulator(0)
accum.value=0
def BFS(graph, source_node):
    chars_list = sc.parallelize([(source_node,0)])
    new_chars = sc.parallelize([source_node])
    while not new_chars.isEmpty():
        level=accum.value
        new_chars=chars_list.filter(lambda x: x[1]==level).join(graph).map(lambda x:[(a,x[1][0]+1) for a in x[1][1]]).flatMap(lambda x:x).distinct()
        accum.add(1)
        chars_list=chars_list.union(new_chars).groupByKey().mapValues(lambda x: min(x))
    return chars_list


In [352]:
node_depths=BFS(linked_chars,'CAPTAIN AMERICA')

In [353]:
node_depths.filter(lambda x: x[1]==3).collect()

[(u'LEONINUS', 3),
 (u'DETHSTRYK', 3),
 (u'24-HOUR MAN/EMMANUEL', 3),
 (u'PINTA', 3),
 (u'DRAGONMEGAS, JESSIE', 3),
 (u'BROTHER AXE', 3),
 (u'OUBLIETTE/', 3),
 (u'SEERESS', 3),
 (u'LOGAN, WAYNE', 3),
 (u'OZ, DR. MALACHI', 3),
 (u'BLACK TALON III', 3),
 (u'MINOTAURUS', 3),
 (u'MARVEL BOY VIII/NOH-', 3),
 (u'CHIMERA/ZUHN', 3),
 (u'AZOPARDI', 3),
 (u'CAPTAIN GLORY', 3),
 (u'PAPA NEBO', 3),
 (u'STILES, TATJANA', 3),
 (u'MOURNING PREY', 3),
 (u'MERREE', 3),
 (u'MASTER FOUR', 3),
 (u'SAINT, AMANDA', 3),
 (u'STORMER', 3),
 (u'SCOTT, JAKE', 3),
 (u'BLOOD-TIDE', 3),
 (u'PLEX INTELLIGENCE', 3),
 (u'ADAMSON, REBECCA', 3),
 (u'COVEY, LAWRENCE K.', 3),
 (u"G'RATH", 3),
 (u'MILLER, JENETTE', 3),
 (u'WARR', 3),
 (u'CHAKRA II', 3),
 (u'KLEIN, SUMMER', 3),
 (u'ADAMSON, JASON', 3),
 (u'NAVARRO', 3),
 (u'DESADIA', 3),
 (u'STRUNK, BELLA', 3),
 (u'DOVINIA', 3)]

In [328]:

def BFS(graph, source_node, depth):
    chars_list = sc.parallelize([(source_node,0)])
    for level in range(depth):
        new_chars=chars_list.filter(lambda x: x[1]==level).join(graph).map(lambda x:[(a,x[1][0]+1) for a in x[1][1]]).flatMap(lambda x:x).distinct()
        if new_chars.isEmpty():
            break
        chars_list=chars_list.union(new_chars).groupByKey().mapValues(lambda x: min(x))
    return chars_list